In [63]:
from support import CalculateLabelsCorrelationWithFTest
from skmultilearn.dataset import load_dataset
from skmultilearn.base.problem_transformation import ProblemTransformationBase
from typing import Any, List
import numpy as np
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC
from evaluation import EvaluationPipeline

In [58]:
full_data = load_dataset("scene", "undivided")
X_full, y_full, _, _ = full_data

scene:undivided - does not exists downloading
Downloaded scene-undivided


In [39]:
def build_chain_based_on_f_test(res):
    chain = []
    sorted_res = res.sort_values(by=["f_test_result"], ascending=False)
    
    element = int(sorted_res.iloc[0]["for_label"])
    chain.append(element)

    m = ~sorted_res["expand_this_label"].isin(chain)
    m &= sorted_res["for_label"] == element
    
    while m.sum() > 0:
        sliced_res = sorted_res[m]
        sorted_sliced_res = sliced_res.sort_values(by=["f_test_result"], ascending=False)

        element = int(sorted_sliced_res.iloc[0]["expand_this_label"])
        chain.append(element)

        m = ~sorted_res["expand_this_label"].isin(chain)
        m &= sorted_res["for_label"] == element
    
    return chain

build_chain_based_on_f_test(res)


[5, 4, 1, 0, 2, 3]

In [73]:
class ClassifierChainWithFTestOrdering(ProblemTransformationBase):
    def __init__(
        self,
        alpha: float = 0.5,
    ):
        self.calculator = CalculateLabelsCorrelationWithFTest(alpha=alpha)
    
    def fit(self, X: Any, y: Any):
        self.classes_ = y.shape[1]

        ordered_chain = build_chain_based_on_f_test(self.calculator.get(y))
        
        self.classifier = ClassifierChain(
            classifier=SVC(),
            require_dense=[False, True],
            order=ordered_chain,
        )

        self.classifier.fit(X, y)
    
    def predict(self, X: Any) -> np.ndarray[Any,Any]:
        return self.classifier.predict(X)


In [75]:
model = ClassifierChainWithFTestOrdering(alpha=1)
# model.fit(X_full, y_full)

In [77]:
pipe = EvaluationPipeline(model)
pipe.run(X_full, y_full)

# TODO: can't use the model with the evaluation pipeline as it is missing some base attrs
# have to check what is going onn